In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = "ruoxinhuang" 
os.environ['KAGGLE_KEY'] = "f9d6d5aade71452503766f80812da73d"
!kaggle competitions download -c 11-785-fall-20-homework-3
!unzip dev.npy.zip
!unzip dev_labels.npy.zip
!unzip phoneme_list.py
!unzip test.npy.zip
!unzip train.npy.zip
!unzip train_labels.npy.zip


  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 51.1MB/s]
 85% 84.0M/98.6M [00:00<00:00, 61.9MB/s]
100% 98.6M/98.6M [00:01<00:00, 95.0MB/s]
  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 94.5MB/s]
 99% 1.56G/1.57G [00:25<00:00, 69.1MB/s]
100% 1.57G/1.57G [00:25<00:00, 66.6MB/s]
 74% 73.0M/98.1M [00:02<00:01, 24.7MB/s]
100% 98.1M/98.1M [00:02<00:00, 45.5MB/s]
  0% 0.00/1.06k [00:00<?, ?B/s]
100% 1.06k/1.06k [00:00<00:00, 660kB/s]
  0% 0.00/16.5k [00:00<?, ?B/s]
100% 16.5k/16.5k [00:00<00:00, 9.49MB/s]
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
Archive:  dev.npy.zip
  inflating: dev.npy                 
Archive:  dev_labels.npy.zip
  inflating: dev_labels.npy          
Archive:  phoneme_list.py
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will b

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
!pip install wget
%cd ctcdecode
!pip install .
%cd ..

Cloning into 'ctcdecode'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 1063 (delta 6), reused 7 (delta 2), pack-reused 1047
Receiving objects: 100% (1063/1063), 763.61 KiB | 1.75 MiB/s, done.
Resolving deltas: 100% (509/509), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 90, done.        
remote: Counting objects: 100% (90/90), done.        
remote: Compressing objects: 100% (64/64), done.        
remote: Total 13672 (delta 41), reused 54 (delta 2

In [ ]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
train = np.load('train.npy', allow_pickle=True, encoding="latin1")
train_labels = np.load('train_labels.npy', allow_pickle=True, encoding="latin1")

dev = np.load('dev.npy', allow_pickle=True, encoding="latin1")
dev_labels = np.load('dev_labels.npy', allow_pickle=True, encoding="latin1")

test = np.load('test.npy', allow_pickle=True, encoding="latin1")

In [ ]:
class MyDataset(Dataset):
  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, index):
    X = self.X[index]
    Y = self.Y[index] + 1
    return torch.from_numpy(X).float(), X.shape[0], torch.from_numpy(Y).long(), Y.shape[0]                

In [ ]:
def collate(batch):
  X = [i[0] for i in batch]
  X_len = [i[1] for i in batch]
  Y = [i[2] for i in batch]
  Y_len = [i[3] for i in batch]

  X_pad = nn.utils.rnn.pad_sequence(X, batch_first=False, padding_value=0.0)
  Y_pad = nn.utils.rnn.pad_sequence(Y, batch_first=True, padding_value=0)


  return X_pad,X_len,Y_pad,Y_len

In [ ]:
train_dataset = MyDataset(train, train_labels)
train_loader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True, collate_fn = collate)
train_loader = DataLoader(train_dataset, **train_loader_args)


dev_dataset = MyDataset(dev, dev_labels)
dev_loader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True, collate_fn = collate)
dev_loader = DataLoader(dev_dataset, **dev_loader_args)

test_dataset = MyDataset(test, train_labels[:len(test)])
test_loader_args = dict(shuffle=False, batch_size=64, num_workers=4, pin_memory=True, collate_fn = collate)
test_loader = DataLoader(test_dataset, **test_loader_args)

In [ ]:
class Model(nn.Module):
    def __init__(self, input, output, hidden_size):
        super(Model, self).__init__()
        self.lstm = nn.LSTM(input, hidden_size, num_layers = 3, bidirectional=True, dropout=0.5)
        self.linear = nn.Linear(hidden_size * 2, hidden_size)
        self.output = nn.Linear(hidden_size, output)
    
    def forward(self, X, lengths):
        packed_X = nn.utils.rnn.pack_padded_sequence(X, lengths, enforce_sorted=False)
        packed_out = self.lstm(packed_X)[0]
        out, out_lens = nn.utils.rnn.pad_packed_sequence(packed_out)
        # Log softmax after output layer is required since`nn.CTCLoss` expects log probabilities.
        out = self.linear(out)
        out = self.output(out).log_softmax(2)
        return out, out_lens

In [ ]:
def train_one_epoch(model, train_loader, optimizer):
    
  model.train()
  total_loss = 0
  count = 0

  for X, X_lens, Y, Y_lens in train_loader:
      X = X.to(device)
      Y = Y.to(device)
      
      optimizer.zero_grad()
      out, out_lens = model(X, X_lens)
      loss = criterion(out, Y, out_lens.tolist(), Y_lens)
      count +=1
      total_loss += loss.item()
      loss.backward()
      optimizer.step()

  print(total_loss/count)

In [ ]:
model = Model(13, 42, 256)
model.to(device)
criterion = nn.CTCLoss()
optimizer = torch.optim.Adam(model.parameters(), weight_decay=5e-6, lr=1e-3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.85)

In [ ]:
for i in range(20):
  train_one_epoch(model, train_loader, optimizer)
  #scheduler.step()

3.700822887032531
3.200249103612678
1.771767225722934
1.0384225077753844
0.8470092453928881
0.7430530613244966
0.6806930636250695
0.6343992612389631
0.598560280578081
0.5741458729256032
0.5526888256156167
0.5224685649885688
0.5122270928046038
0.5033984678949035
0.5020077262333659
0.47501810241577236
0.46618958519295206
0.4540723036368226
0.44034592530062033
0.42916270640007287


In [ ]:
from ctcdecode import CTCBeamDecoder
from phoneme_list import PHONEME_MAP

decoder = CTCBeamDecoder(PHONEME_MAP, beam_width=10, log_probs_input=True)

In [ ]:
from nltk.metrics.distance import edit_distance
model.eval()
torch.no_grad()
distance = 0
total_loss = 0
count = 0

for X, X_lens, Y, Y_lens in dev_loader:
  X = X.to(device)
  out, out_lens = model(X, X_lens)
  loss = criterion(out, Y, out_lens.tolist(), Y_lens)
  count +=1
  total_loss += loss.item()
  beam, _, _, out_lens = decoder.decode(torch.transpose(out,0,1),out_lens)
  beam = beam[:,0]
  out_lens = out_lens[:,0]


  for i, lengths in enumerate(out_lens):
    output = beam[i][:lengths]
    label = ''.join([PHONEME_MAP[idx] for idx in Y[i][:Y_lens[i]]])
    output = ''.join([PHONEME_MAP[idx] for idx in output])
    distance += edit_distance(output,label)
print(total_loss/count)

0.5290674647769412


In [ ]:
distance/len(dev_dataset)

10.634648370497427

In [ ]:
result = []
model.eval()
torch.no_grad()

for X, X_lens, Y, Y_lens in test_loader:
  X = X.to(device)
  out, out_lens = model(X, X_lens)
  beam, _, _, out_lens = decoder.decode(torch.transpose(out,0,1),out_lens)
  beam = beam[:,0]
  out_lens = out_lens[:,0]

  for i, lengths in enumerate(out_lens):
    output = beam[i][:lengths]
    output = ''.join([PHONEME_MAP[idx] for idx in output])
    result.append(output)

In [ ]:
import csv

with open('submission4.csv', 'w') as csvfile:
  csvwriter = csv.writer(csvfile)
  csvwriter.writerow(('id','label'))
  for i, row in enumerate(result):
    csvwriter.writerow((i,row))